<a href="https://colab.research.google.com/github/bill7845/brunch_networking_aws/blob/main/Notebook/brunch_crawling_p1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# category_dict = {
#     '오늘은_이런_책?q=g' : 7, '뮤직_인사이드?q=g' : 8,
#     '직장인_현실_조언?q=g' : 10, '스타트업_경험담?q=g' : 11, '육아_이야기?q=g' : 12,
#     '요리·레시피?q=g' : 13, '건강·운동?q=g' : 14, '멘탈_관리_심리_탐구?q=g' :15,
#     '문화·예술?q=g' : 17, '인문학·철학?q=g' : 19, '쉽게_읽는_역사?q=g' : 20,
#     '우리집_반려동물?q=g' : 21, '사랑·이별?q=g' : 23, '감성_에세이?q=g' : 24
#     }

In [ ]:
# library import
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import NoAlertPresentException
from selenium.webdriver.chrome.options import Options
import time
import requests
import pickle
from bs4 import BeautifulSoup

## 카테고리 별 게시글 리스트 페이지에서 유저별 id 파싱 (*전체 18개 카테고리)
## 각 페이지별 무한 스크롤 javaScript 제어를 위한 셀레니움 기능 사용
def get_user_id(BASE_URL):
        
    chromedriver = 'C:/chrome_driver/chromedriver'
#     options = webdriver.ChromeOptions()
#     options.add_argument('headless')
#     options.add_argument('window-size=1920x1080')
    
    category_dict = {
        '감성_에세이?q=g' : 24
    }
    
    for category in category_dict:
        idx = category_dict[category]
        val = category
        
        driver = webdriver.Chrome(chromedriver) # , options=options
        driver.implicitly_wait(3)
        driver.get(BASE_URL)
        
        xpath = "//*[@id='mArticle']/div[3]/div[1]/div/a[{}]".format(idx)
        driver.find_element_by_xpath(xpath).click()
        driver.switch_to_window(driver.window_handles[1])
        driver.get_window_position(driver.window_handles[1])
        
        elem = driver.find_element_by_tag_name('body')
        
        while True :
            for i in range(100):
                elem.send_keys(Keys.END)
                time.sleep(3)
            publish_time = driver.find_elements_by_class_name('publish_time')
            if  publish_time[-15].text[-4:] == "2019":
                print("break point ",publish_time[-15].text)
                break
            else :
                print("continue ",publish_time[-15].text)
                continue
                
                
        html = driver.page_source
        soup = BeautifulSoup(html,'html.parser')

        # 게시글별 user_id parsing
        a_tags = soup.select('#wrapArticle > div.wrap_article_list.\#keyword_related_contents > ul > li > a.link_post')

        user_id = []
        for a_tag in a_tags :
            user_id.append(a_tag['href'])
            
        with open(category[:-4]+'_userId.txt','wb') as f:
            pickle.dump(user_id,f)
        print("complete ",category)
        print("###################")
                       
        driver.quit()
        
BASE_URL = "https://brunch.co.kr"
get_user_id(BASE_URL)